In [1]:
import pandas as pd
df = pd.read_csv("data/application_record.csv")


In [5]:
df.isnull().sum()


ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
dtype: int64

In [3]:
import pandas as pd

mode_value = df['OCCUPATION_TYPE'].mode()[0]

# Replace missing values with the mode
df['OCCUPATION_TYPE'].fillna(mode_value, inplace=True)

# Verify that missing values have been filled
print(df['OCCUPATION_TYPE'].isnull().sum())  # Should print 0 if all missing values are replaced


0


In [4]:
import numpy as np

def aggregate_columns(column):
    if column.dtype == 'O':  # If categorical
        # Check if the mode exists
        mode = column.mode()
        if not mode.empty:
            return mode.iloc[0]  # Most frequent value
        else:
            return None  # Return None if no mode is found
    else:  # For numerical columns
        return column.mean()  # Use mean

df = df.groupby('ID').agg(aggregate_columns).reset_index()
print(df)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y           0.0   
1       5008805           M            Y               Y           0.0   
2       5008806           M            Y               Y           0.0   
3       5008808           F            N               Y           0.0   
4       5008809           F            N               Y           0.0   
...         ...         ...          ...             ...           ...   
438505  7999660           F            N               N           0.0   
438506  7999696           F            N               Y           2.0   
438507  7999738           M            N               Y           0.0   
438508  7999784           F            Y               Y           1.0   
438509  7999952           F            N               Y           1.0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

In [6]:
df.drop(columns=['FLAG_MOBIL'], inplace=True)

print(df.columns)

Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS'],
      dtype='object')


In [7]:
# Convert negative values in 'DAYS_BIRTH' and 'DAYS_EMPLOYED' to positive values
df['DAYS_BIRTH'] = abs(df['DAYS_BIRTH'])
df['DAYS_EMPLOYED'] = abs(df['DAYS_EMPLOYED'])

# Optionally, if you want to convert days to years (for 'DAYS_BIRTH'):
df['AGE'] = df['DAYS_BIRTH'] / 365  # Convert days to years

# Optionally, if you want to convert 'DAYS_EMPLOYED' to years:
df['EMPLOYMENT_YEARS'] = df['DAYS_EMPLOYED'] / 365  # Convert days to years

# Verify changes
print(df[['DAYS_BIRTH', 'AGE', 'DAYS_EMPLOYED', 'EMPLOYMENT_YEARS']].tail())


        DAYS_BIRTH        AGE  DAYS_EMPLOYED  EMPLOYMENT_YEARS
438505     13432.0  36.800000         5446.0         14.920548
438506     12576.0  34.454795         4382.0         12.005479
438507      9970.0  27.315068          119.0          0.326027
438508     10630.0  29.123288          454.0          1.243836
438509     15859.0  43.449315         3679.0         10.079452


In [8]:
# Handling extreme negative values (optional)
df.loc[df['DAYS_EMPLOYED'] < -10000, 'DAYS_EMPLOYED'] = None  # Mark extreme negative values as missing (e.g., > 10 years)


In [9]:
import numpy as np

# Assuming df is your DataFrame
# Ensure that the 'DAYS_BIRTH' and 'DAYS_EMPLOYED' columns are already in the correct format (days)

# Convert 'DAYS_BIRTH' to age in years
df['AGE'] = abs(df['DAYS_BIRTH']) / 365

# Convert 'DAYS_EMPLOYED' to employment years
df['EMPLOYMENT_YEARS'] = abs(df['DAYS_EMPLOYED']) / 365

# Handle the case where employment years are greater than the person's age minus 18
df.loc[df['EMPLOYMENT_YEARS'] > (df['AGE'] - 18), 'EMPLOYMENT_YEARS'] = np.nan  # Replace with NaN if unrealistic

# Handle extreme employment years (e.g., more than 100 years of employment)
df.loc[df['EMPLOYMENT_YEARS'] > 100, 'EMPLOYMENT_YEARS'] = np.nan  # Replace with NaN for extreme values

# Verify the changes
print(df[['DAYS_BIRTH', 'AGE', 'DAYS_EMPLOYED', 'EMPLOYMENT_YEARS']].tail())


        DAYS_BIRTH        AGE  DAYS_EMPLOYED  EMPLOYMENT_YEARS
438505     13432.0  36.800000         5446.0         14.920548
438506     12576.0  34.454795         4382.0         12.005479
438507      9970.0  27.315068          119.0          0.326027
438508     10630.0  29.123288          454.0          1.243836
438509     15859.0  43.449315         3679.0         10.079452


In [10]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame and 'AMT_INCOME_TOTAL' is the column to normalize

scaler = MinMaxScaler()

# Normalize the 'AMT_INCOME_TOTAL' column
df['AMT_INCOME_TOTAL'] = scaler.fit_transform(df[['AMT_INCOME_TOTAL']])
df['AGE'] = scaler.fit_transform(df[['AGE']])
df['EMPLOYMENT_YEARS'] = scaler.fit_transform(df[['EMPLOYMENT_YEARS']])

# Verify the result
print(df[['AMT_INCOME_TOTAL']].tail())


        AMT_INCOME_TOTAL
438505          0.009503
438506          0.016196
438507          0.022889
438508          0.022889
438509          0.019542


In [11]:
# Check for missing values
print(df.isnull().sum())


ID                         0
CODE_GENDER                0
FLAG_OWN_CAR               0
FLAG_OWN_REALTY            0
CNT_CHILDREN               0
AMT_INCOME_TOTAL           0
NAME_INCOME_TYPE           0
NAME_EDUCATION_TYPE        0
NAME_FAMILY_STATUS         0
NAME_HOUSING_TYPE          0
DAYS_BIRTH                 0
DAYS_EMPLOYED              0
FLAG_WORK_PHONE            0
FLAG_PHONE                 0
FLAG_EMAIL                 0
OCCUPATION_TYPE            0
CNT_FAM_MEMBERS            0
AGE                        0
EMPLOYMENT_YEARS       75574
dtype: int64


In [12]:
# Get descriptive statistics for the 'EMPLOYMENT_YEARS' column
print(df['EMPLOYMENT_YEARS'].describe()) #DATA IS RIGHT SKEWED USE MEDIAN


count    362936.000000
mean          0.148901
std           0.136842
min           0.000000
25%           0.051487
50%           0.109082
75%           0.199612
max           1.000000
Name: EMPLOYMENT_YEARS, dtype: float64


In [13]:
df['EMPLOYMENT_YEARS'] = df['EMPLOYMENT_YEARS'].fillna(df['EMPLOYMENT_YEARS'].median())

In [14]:
# Check for missing values
print(df.isnull().sum())


ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
AGE                    0
EMPLOYMENT_YEARS       0
dtype: int64


In [16]:
# Check the range of numeric features
print(df[['AGE', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH']].min())
print(df[['AGE', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH']].max())


AGE                    0.0
AMT_INCOME_TOTAL       0.0
DAYS_BIRTH          7489.0
dtype: float64
AGE                     1.0
AMT_INCOME_TOTAL        1.0
DAYS_BIRTH          25201.0
dtype: float64


In [17]:
# Check if new features are correctly created (e.g., 'AGE' or 'EMPLOYMENT_YEARS')
print(df[['AGE', 'EMPLOYMENT_YEARS']].head())


        AGE  EMPLOYMENT_YEARS
0  0.254968          0.258576
1  0.254968          0.258576
2  0.789578          0.064045
3  0.656109          0.173469
4  0.656109          0.173469


In [18]:
# Filter rows where AMT_INCOME_TOTAL is 0 or negative
invalid_income = df[df['AMT_INCOME_TOTAL'] <= 0]

# Display the invalid rows to inspect them
print(invalid_income)


             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
238132  6093712           F            N               N           0.0   
238133  6093713           F            N               N           0.0   

        AMT_INCOME_TOTAL NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
238132               0.0        Pensioner  Secondary / secondary special   
238133               0.0        Pensioner  Secondary / secondary special   

       NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
238132            Married  House / apartment     21003.0       365243.0   
238133            Married  House / apartment     21003.0       365243.0   

        FLAG_WORK_PHONE  FLAG_PHONE  FLAG_EMAIL OCCUPATION_TYPE  \
238132              0.0         0.0         0.0        Laborers   
238133              0.0         0.0         0.0        Laborers   

        CNT_FAM_MEMBERS       AGE  EMPLOYMENT_YEARS  
238132              2.0  0.762986          0.109082  
238133      

In [19]:
# Drop rows where AMT_INCOME_TOTAL is 0
df = df[df['AMT_INCOME_TOTAL'] != 0]

# Check the number of rows remaining
print(f"Number of rows after dropping zero income: {df.shape[0]}")


Number of rows after dropping zero income: 438508


In [20]:
# Filter rows where AMT_INCOME_TOTAL is 0 or negative
invalid_income = df[df['AMT_INCOME_TOTAL'] <= 0]

# Display the invalid rows to inspect them
print(invalid_income)

Empty DataFrame
Columns: [ID, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, CNT_CHILDREN, AMT_INCOME_TOTAL, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, DAYS_BIRTH, DAYS_EMPLOYED, FLAG_WORK_PHONE, FLAG_PHONE, FLAG_EMAIL, OCCUPATION_TYPE, CNT_FAM_MEMBERS, AGE, EMPLOYMENT_YEARS]
Index: []


In [23]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Optionally, remove duplicates
df = df.drop_duplicates()
# Check for negative or unrealistic values in 'DAYS_BIRTH' (age)
print(f"Negative age values: {df[df['DAYS_BIRTH'] < 0].shape[0]}")

# Check for extreme or unrealistic values in 'DAYS_EMPLOYED'
print(f"Extreme 'DAYS_EMPLOYED' values: {df[df['DAYS_EMPLOYED'] > 365243].shape[0]}")  # 1000+ years
# Check for zero income values
zero_income = df[df['AMT_INCOME_TOTAL'] == 0]
print(f"Rows with zero income: {zero_income.shape[0]}")


Number of duplicate rows: 0
Negative age values: 0
Extreme 'DAYS_EMPLOYED' values: 0
Rows with zero income: 0


In [24]:
df.to_csv('data/application_record_cleaned.csv', index=False)

# If you are running in a web-based or GUI framework, implement download functionality accordingly.
print("File has been saved as 'application_record_cleaned.csv'. You can provide this file for download if needed.")

File has been saved as 'application_record_cleaned.csv'. You can provide this file for download if needed.
